In [1]:
# CutQC demo: 3-qubit GHZ circuit, single cut on qubit-1 (between the two CNOTs)


from qiskit import QuantumCircuit
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_aer import AerSimulator
from qiskit.transpiler import generate_preset_pass_manager
import numpy as np

In [2]:
backend = AerSimulator()
sampler = Sampler(mode=backend)

# Initial circuit (GHZ-style entanglement)
qc_base = QuantumCircuit(2)
qc_base.h(0)
qc_base.cx(0, 1)

# Basis list for qubit 1 only
basis_list = ["X", "Y", "Z"]
results_counts = {}
results = {}

for basis in basis_list:
    qc = qc_base.copy()

    # Rotate qubit 1 to the target basis before measurement
    if basis == "X":
        qc.h(1)
    elif basis == "Y":
        qc.sdg(1)
        qc.h(1)
    # For "I" or "Z", no gate is needed

    qc.measure_all()

    # Transpile for FakeKolkata ISA
    #pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    #transpiled_qc = pm.run(qc)

    # Run with Sampler
    job = sampler.run([qc], shots=8192)
    result = job.result()
    counts = result[0].data.meas.get_counts()

    # Convert counts to probabilities
    total_shots = sum(counts.values())
    probs_basis = {k: v / total_shots for k, v in counts.items()}

    results_counts[basis] = counts

    results[basis] = probs_basis

    # Store results
    

# Print all results
for basis, counts in results_counts.items():
    print(f"Measurement in {basis} basis → {counts}")


# Print all results as probabilities
for basis, probs_basis in results.items():
    print(f"\nMeasurement Probabilities in {basis} basis:")
    for outcome, prob in sorted(probs_basis.items()):
        print(f"  {outcome} → {prob:.4f}")


Measurement in X basis → {'00': 1963, '10': 2044, '11': 2117, '01': 2068}
Measurement in Y basis → {'00': 2186, '11': 2055, '10': 1996, '01': 1955}
Measurement in Z basis → {'11': 4146, '00': 4046}

Measurement Probabilities in X basis:
  00 → 0.2396
  01 → 0.2524
  10 → 0.2495
  11 → 0.2584

Measurement Probabilities in Y basis:
  00 → 0.2668
  01 → 0.2386
  10 → 0.2437
  11 → 0.2509

Measurement Probabilities in Z basis:
  00 → 0.4939
  11 → 0.5061


In [3]:
import json

# Convert values to float (JSON doesn't support numpy types well)
basis_data = {
    prep: {k: float(v) for k, v in probs_basis.items()}
    for prep, probs_basis in results.items()
}

with open("basis_results.json", "w") as f:
    json.dump(basis_data, f, indent=4)

In [4]:
prep_list = ['|0>', '|1>', '|+>', '|+i>']
results_counts_prep = {}
results_prob_prep = {}

for prep in prep_list:
    qc2 = QuantumCircuit(2)

    # Prepare qubit 0 in the desired eigen-state
    if prep == '|1>':
        qc2.x(0)
    elif prep == '|+>':
        qc2.h(0)
    elif prep == '|+i>':
        qc2.h(0)
        qc2.s(0)
    # if prep == '|0>', do nothing (default |0>)

    qc2.cx(0, 1)  # creates entanglement, completes GHZ

    qc2.measure_all()

    # Run with Sampler
    job = sampler.run([qc2], shots=8192)
    result = job.result()
    counts = result[0].data.meas.get_counts()

    # Convert counts to probabilities
    total_shots = sum(counts.values())
    probs_prep_val = {k: v / total_shots for k, v in counts.items()}

    #results_counts[basis] = counts

    results_prob_prep[prep] = probs_prep_val

    # Store results
    results_counts_prep[prep] = counts

# Print all results
for prep, counts_prep in results_counts_prep.items():
    print(f"Measurement with preparation {prep} → {counts_prep}")


# Print all results as probabilities
for prep, probs_prep in results_prob_prep.items():
    print(f"\nWhen the initilaization is in {prep} eigenstate : ")
    for outcome_prep, prob_prep in sorted(probs_prep.items()):
        print(f"  {outcome_prep} → {prob_prep:.4f}")


Measurement with preparation |0> → {'00': 8192}
Measurement with preparation |1> → {'11': 8192}
Measurement with preparation |+> → {'00': 4145, '11': 4047}
Measurement with preparation |+i> → {'11': 4059, '00': 4133}

When the initilaization is in |0> eigenstate : 
  00 → 1.0000

When the initilaization is in |1> eigenstate : 
  11 → 1.0000

When the initilaization is in |+> eigenstate : 
  00 → 0.5060
  11 → 0.4940

When the initilaization is in |+i> eigenstate : 
  00 → 0.5045
  11 → 0.4955


In [5]:
import json

# Convert values to float (JSON doesn't support numpy types well)
prep_data = {
    prep: {k: float(v) for k, v in probs_prep_val.items()}
    for prep, probs_prep_val in results_prob_prep.items()
}

with open("prep_results.json", "w") as f:
    json.dump(prep_data, f, indent=4)

In [6]:
input_strings_subckt1 = ['0', '1']
input_strings_subckt2 = ['00','01','10','11']

In [7]:
import json

# Load basis_results from JSON file
with open("basis_results.json", "r") as f:
    basis_results = json.load(f)

results_array_subckt1 = []
for input_val in input_strings_subckt1:
    if input_val == '0':
        #z_00 = basis_results.get("Z", {}).get("00", 0)
        #z_01 = basis_results.get("Z", {}).get("01", 0)

        p11 = basis_results.get("Z", {}).get("00", 0) + basis_results.get("Z", {}).get("01", 0) + basis_results.get("Z", {}).get("00", 0) - basis_results.get("Z", {}).get("01", 0)
        p12 = basis_results.get("Z", {}).get("00", 0) + basis_results.get("Z", {}).get("01", 0) - basis_results.get("Z", {}).get("00", 0) + basis_results.get("Z", {}).get("01", 0)
        p13 = basis_results.get("X", {}).get("00", 0) - basis_results.get("X", {}).get("01", 0)
        p14 = basis_results.get("Y", {}).get("00", 0) - basis_results.get("Y", {}).get("01", 0)

    elif input_val == '1':
        #z_10 = basis_results.get("Z", {}).get("10", 0)
        #z_11 = basis_results.get("Z", {}).get("11", 0)

        p11 = basis_results.get("Z", {}).get("10", 0) + basis_results.get("Z", {}).get("11", 0) + basis_results.get("Z", {}).get("10", 0) - basis_results.get("Z", {}).get("11", 0)
        p12 = basis_results.get("Z", {}).get("10", 0) + basis_results.get("Z", {}).get("11", 0) - basis_results.get("Z", {}).get("10", 0) + basis_results.get("Z", {}).get("11", 0)
        p13 = basis_results.get("X", {}).get("10", 0) - basis_results.get("X", {}).get("11", 0)
        p14 = basis_results.get("Y", {}).get("10", 0) - basis_results.get("Y", {}).get("11", 0)

    # Print results for each input
    results_array_subckt1.append({
    "Input for Sub-Circuit 1 is": input_val,
        "p11": p11,
        "p12": p12,
        "p13": p13,
        "p14": p14
    })
print(results_array_subckt1)

[{'Input for Sub-Circuit 1 is': '0', 'p11': 0.98779296875, 'p12': 0.0, 'p13': -0.0128173828125, 'p14': 0.0281982421875}, {'Input for Sub-Circuit 1 is': '1', 'p11': 0.0, 'p12': 1.01220703125, 'p13': -0.0089111328125, 'p14': -0.0072021484375}]


In [8]:
import json

# Load basis_results from JSON file
with open("prep_results.json", "r") as f:
    prep_results = json.load(f)

results_array_subckt2 = []
for input_val in input_strings_subckt2:
    if input_val == '00':
        #z_00 = basis_results.get("Z", {}).get("00", 0)
        #z_01 = basis_results.get("Z", {}).get("01", 0)

        p21 = prep_results.get("|0>", {}).get("00", 0)
        p22 = prep_results.get("|1>", {}).get("00", 0)
        p23 = 2 * prep_results.get("|+>", {}).get("00", 0) - prep_results.get("|0>", {}).get("00", 0) - prep_results.get("|1>", {}).get("00", 0)
        p24 = 2 * prep_results.get("|+i>", {}).get("00", 0) - prep_results.get("|0>", {}).get("00", 0) - prep_results.get("|1>", {}).get("00", 0)

    elif input_val == '01':
        #z_10 = basis_results.get("Z", {}).get("10", 0)
        #z_11 = basis_results.get("Z", {}).get("11", 0)
        p21 = prep_results.get("|0>", {}).get("01", 0)
        p22 = prep_results.get("|1>", {}).get("01", 0)
        p23 = 2 * prep_results.get("|+>", {}).get("01", 0) - prep_results.get("|0>", {}).get("01", 0) - prep_results.get("|1>", {}).get("01", 0)
        p24 = 2 * prep_results.get("|+i>", {}).get("01", 0) - prep_results.get("|0>", {}).get("01", 0) - prep_results.get("|1>", {}).get("01", 0)
        

    elif input_val == '10':
        #z_10 = basis_results.get("Z", {}).get("10", 0)
        #z_11 = basis_results.get("Z", {}).get("11", 0)

        p21 = prep_results.get("|0>", {}).get("10", 0)
        p22 = prep_results.get("|1>", {}).get("10", 0)
        p23 = 2 * prep_results.get("|+>", {}).get("10", 0) - prep_results.get("|0>", {}).get("10", 0) - prep_results.get("|1>", {}).get("10", 0)
        p24 = 2 * prep_results.get("|+i>", {}).get("10", 0) - prep_results.get("|0>", {}).get("10", 0) - prep_results.get("|1>", {}).get("10", 0)

    elif input_val == '11':
        #z_10 = basis_results.get("Z", {}).get("10", 0)
        #z_11 = basis_results.get("Z", {}).get("11", 0)
        p21 = prep_results.get("|0>", {}).get("11", 0)
        p22 = prep_results.get("|1>", {}).get("11", 0)
        p23 = 2 * prep_results.get("|+>", {}).get("11", 0) - prep_results.get("|0>", {}).get("11", 0) - prep_results.get("|1>", {}).get("11", 0)
        p24 = 2 * prep_results.get("|+i>", {}).get("11", 0) - prep_results.get("|0>", {}).get("11", 0) - prep_results.get("|1>", {}).get("11", 0)

    # Print results for each input
    results_array_subckt2.append({
    "Input for Sub-Circuit 2 is": input_val,
        "p21": p21,
        "p22": p22,
        "p23": p23,
        "p24": p24
    })
print(results_array_subckt2)

[{'Input for Sub-Circuit 2 is': '00', 'p21': 1.0, 'p22': 0, 'p23': 0.011962890625, 'p24': 0.009033203125}, {'Input for Sub-Circuit 2 is': '01', 'p21': 0, 'p22': 0, 'p23': 0, 'p24': 0}, {'Input for Sub-Circuit 2 is': '10', 'p21': 0, 'p22': 0, 'p23': 0, 'p24': 0}, {'Input for Sub-Circuit 2 is': '11', 'p21': 0, 'p22': 1.0, 'p23': -0.011962890625, 'p24': -0.009033203125}]


In [9]:
final_results = [] # Final results will be stored here

for res1 in results_array_subckt1:
    for res2 in results_array_subckt2:
        product_sum = (
            res1['p11'] * res2['p21'] +
            res1['p12'] * res2['p22'] +
            res1['p13'] * res2['p23'] +
            res1['p14'] * res2['p24']
        )

        final_results.append({
            " For Sub-Circuit 1 Input": res1['Input for Sub-Circuit 1 is'],
            "For Sub-Circuit 2 Input": res2['Input for Sub-Circuit 2 is'],
            "Final Value for the probability is": (product_sum/2)
        })

#print(final_results)
for result in final_results:
    print(result)

{' For Sub-Circuit 1 Input': '0', 'For Sub-Circuit 2 Input': '00', 'Final Value for the probability is': 0.49394717812538147}
{' For Sub-Circuit 1 Input': '0', 'For Sub-Circuit 2 Input': '01', 'Final Value for the probability is': 0.0}
{' For Sub-Circuit 1 Input': '0', 'For Sub-Circuit 2 Input': '10', 'Final Value for the probability is': 0.0}
{' For Sub-Circuit 1 Input': '0', 'For Sub-Circuit 2 Input': '11', 'Final Value for the probability is': -5.0693750381469727e-05}
{' For Sub-Circuit 1 Input': '1', 'For Sub-Circuit 2 Input': '00', 'Final Value for the probability is': -8.58306884765625e-05}
{' For Sub-Circuit 1 Input': '1', 'For Sub-Circuit 2 Input': '01', 'Final Value for the probability is': 0.0}
{' For Sub-Circuit 1 Input': '1', 'For Sub-Circuit 2 Input': '10', 'Final Value for the probability is': 0.0}
{' For Sub-Circuit 1 Input': '1', 'For Sub-Circuit 2 Input': '11', 'Final Value for the probability is': 0.5061893463134766}
